In [ ]:
import pandas as pd

In [ ]:
import pandas as pd
import urllib.request
import datetime as dt

def fetch_from_unitime(day_diff, filename="data.csv", start_hour=8, end_hour=22, min_size=0, max_size=500):
    if filename[-4:] != ".csv":
        raise Exception("file must be a csv file")
    if day_diff < 0:
        raise Exception("Day difference must be non-negative")
    if day_diff == 0:
        if start_hour > end_hour:
            raise Exception("Start time must be before end time")
    if start_hour > 23 or end_hour > 23:
        raise Exception("hour must be less than 24")
    if min_size > max_size:
        raise Exception("max_size must be greater than min_size")
    if min_size < 0 or max_size < 0:
        raise Exception("Class size must be non-negative")
    current_time = dt.datetime.today()
    time_delta = dt.datetime.today() + dt.timedelta(days=day_diff)


    link = "https://timetable.mypurdue.purdue.edu/Timetabling/export?output=events.csv&type=room&term=Spring2023PWL&e:from={0:02d}/{1:02d}/{2}&e:to={3:02d}/{4:02d}/{5}&e:after={6}00&e:before={7}00&r:type=genClassroom&r:size={8}..{9}".format(current_time.month, current_time.day, current_time.year, time_delta.month, time_delta.day, time_delta.year, start_hour, end_hour, min_size, max_size)
    urllib.request.urlretrieve(link, '/Users/seanlee/Library/CloudStorage/OneDrive-purdue.edu/2/307/Odyssey/classdata/%s'%filename)

def clean_file(df):
    df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)
    for i in range(len(df)):
        start = df.at[i, 'Published Start']
        if start[-1] == 'a' or start[0:2] == '12':
            if len(start.split(':')[0]) == 1:
                df.at[i, 'Published Start'] = '0%s' %(start[:-1])
            else:
                df.at[i, 'Published Start'] = start[:-1]
        elif start == 'noon':
            df.at[i, 'Published Start'] = '12:00'
        else:
            hour = int(start.split(':')[0])
            hour += 12
            rest = start.split(':')[1]
            df.at[i, 'Published Start'] = '%d:%s' %(hour,rest[0:2])


        end = df.at[i, 'Published End']
        if end[-1] == 'a' or end[0:2] == '12':
            if len(end.split(':')[0]) == 1:
                df.at[i, 'Published End'] = '0%s' %(end[:-1])
            else:
                df.at[i, 'Published End'] = end[:-1]
        elif end == 'noon':
            df.at[i, 'Published End'] = '12:00'
        else:
            hour = int(end.split(':')[0])
            hour += 12
            rest = end.split(':')[1]
            df.at[i, 'Published End'] = '%d:%s' %(hour,rest[0:2])

def find_rooms(df):
    building = dict()
    for room in df["Location"].unique():
        if building.get(room.split()[0], 0) == 0:
            building[room.split()[0]] = [room.split()[1]]
        else:
            building[room.split()[0]].append(room.split()[1])
    return building

In [ ]:
df = pd.read_csv("friday.csv")

In [ ]:
clean_file(df)

In [ ]:
len(df)

In [ ]:
for date in set(df["Date"]):
    for classroom in set(df["Location"]):
        df.loc[len(df)] = [pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, date, '06:00', '6:00', classroom, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA]
        df.loc[len(df)] = [pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, date, '22:00', '22:00', classroom, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA]

In [ ]:
len(df)

In [ ]:
len(set(df["Date"]))

In [ ]:
len(set(df["Location"]))

In [ ]:
sorted_df = df.sort_values("Published Start")

In [ ]:
time = dt.datetime.strptime('06:00' + " " + "03/03/2023", '%H:%S %d/%m/%Y')

In [ ]:
thing = pd.DataFrame(columns=["one", "two"])

In [ ]:
thing.loc[len(thing)] = ["hi", "bye"]

In [ ]:
date_list = []
for d_index, date in enumerate(sorted(list(set(df["Date"])))):
    class_list = [[] for i in range(len(sorted_df))]
    for c_index, classroom in enumerate(sorted(list(set(sorted_df["Location"])))):
        class_list[c_index] = sorted_df[(sorted_df["Location"] == classroom) & (sorted_df["Date"] == date)].reset_index()
    date_list.append(class_list)

In [ ]:
sorted_df.columns

In [ ]:
import json
with open('build_address.json') as json_file:
    build_address = json.load(json_file)

In [ ]:
len(build_address)

In [ ]:
build_address

In [ ]:
'HAMP' in build_address

In [ ]:
temp = pd.DataFrame(columns=['Date','Start','End', 'Location', 'Capacity', 'Address'])
for d_index, date in enumerate(sorted(list(set(df["Date"])))):
    for c_index, classroom in enumerate(sorted(list(set(sorted_df["Location"])))):
        for i in range(len(date_list[d_index][c_index])):
            if i == 0:
                continue
            temp.loc[len(temp)] = [date, date_list[d_index][c_index].at[i - 1, "Published End"], date_list[d_index][c_index].at[i, "Published Start"], classroom, date_list[d_index][c_index].at[1, "Capacity"], build_address[classroom.split()[0]]]

In [ ]:
temp.head(20)

In [ ]:
temp.to_csv("friday_open.csv", index=False)

In [ ]:
date_dict = {}

    class_dict = {}
    
        for i in range(len(date_list[d_index][c_index])):
            if i == 0:
                continue
            temp.loc[len(temp)] = [date, date_list[d_index][c_index].at[i - 1, "Published End"], date_list[d_index][c_index].at[i, "Published Start"], classroom, date_list[d_index][c_index].at[1, "Capacity"]]
        class_dict[classroom] = temp.to_json()
    date_dict[date] = class_dict


In [ ]:
date_dict["03/03/2023"]['LWSN B151']

In [ ]:
import json

In [ ]:
with open("/Users/seanlee/Library/CloudStorage/OneDrive-purdue.edu/2/307/Odyssey/classdata/friday.json", "w") as outfile:
    json.dump(date_dict, outfile)

In [ ]:
df2 = pd.read_json("/Users/seanlee/Library/CloudStorage/OneDrive-purdue.edu/2/307/Odyssey/classdata/open_time.ipynb")

In [ ]:
for date in date_dict:
    

In [ ]:
date_dict["03/03/2023"]['ABE B053']

In [ ]:
date_dict['03/03/2023']['ABE B053']

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/Users/seanlee/Library/CloudStorage/OneDrive-purdue.edu/2/307/Odyssey/backend/friday_open.csv")

In [ ]:
df.drop(['Date'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
for i in range(len(df)):
    df.at[i, "building"] = df.at[i, "Location"].split()[0]

In [ ]:
for i in range(len(df)):
    df.at[i, "room"] = df.at[i, "Location"].split()[1]

In [ ]:
for i in range(len(df)):
    df.at[i, "room"] = df.at[i, "Location"].split()[1]

In [ ]:
df.head()

In [ ]:
df.head()

In [ ]:
df.drop("Capacity", axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
arr = df.to_numpy()

In [ ]:
arr[0][0]

In [ ]:
arr[0][0].split(':')[0]

In [ ]:
for i in range(len(df)):
    end = int(arr[i, 1].split(':')[0]) * 60 + int(arr[i, 1].split(':')[1])
    start = int(arr[i, 0].split(':')[0]) * 60 + int(arr[i, 0].split(':')[1])
    if end < 990 or start > 930:
        arr[i][0] = "bye"

In [ ]:
df2 = pd.DataFrame(arr, columns=["Start", "End", "building", "room"])

In [ ]:
df2.head()

In [ ]:
df2.to_csv("open_friday5.csv", index=False, header=None)

In [ ]:
import json

In [ ]:
with open("build_address.json") as f_in:
    my_dict = json.load(f_in)

In [ ]:
my_dict.keys()

In [ ]:
set(df2["building"])

In [ ]:
my_dict.keys() - set(df2["building"])

In [ ]:
my_dict.keys()

In [ ]:
len(df2)

In [ ]:
df2.head()

In [ ]:
df2 = df2.drop(df2[df2.Start == "bye"].index)

In [ ]:
df2 = df2.drop(df2[df2.Start == df2.End].index)

In [ ]:
df2 = df2.drop(df2[(df2.End.split(':')[0] * 60 + df2.End.split(':')[1]) - (df2.Start.split(':')[0] * 60 + df2.End.split(':')[1]) < 60].index)

In [ ]:
arr2 = df2.to_numpy()

In [ ]:
i = 3
end = int(arr2[i, 1].split(':')[0]) * 60 + int(arr2[i, 1].split(':')[1])
start = int(arr2[i, 0].split(':')[0]) * 60 + int(arr2[i, 0].split(':')[1])
if end - start < 60:
    arr[i][0] = "bye"

In [ ]:
arr[i][0]

In [ ]:
for i in range(len(arr2)):
    end = int(arr2[i, 1].split(':')[0]) * 60 + int(arr2[i, 1].split(':')[1])
    start = int(arr2[i, 0].split(':')[0]) * 60 + int(arr2[i, 0].split(':')[1])
    if end - start < 60:
        arr2[i][0] = "bye"

In [ ]:
df2 = pd.DataFrame(arr2, columns=["Start", "End", "building", "room"])

In [ ]:
df2.head()

In [ ]:
df2 = df2.drop(df2[df2.Start == "bye"].index)

In [ ]:
len(df2)

In [ ]:
df2.to_csv("open_friday4.csv", index=False, header=None)

In [ ]:
df2.head(50)

In [ ]:
df2.head(50)

In [ ]:
len(df2)

In [ ]:
df2.head()

In [ ]:
df2.to_csv("open_friday3.csv", index=False)

In [ ]:
df.to_csv("open_friday2.csv", index=False)

In [ ]:
df.to_csv("open_friday2.csv", index=False)

In [ ]:
thing = df.to_dict()

In [ ]:
thing = df[df["Start"].str.split(':')[0] > 15]

In [ ]:
for i in range(df):
    df.at[i, "Start"] = df.at[i, "Start"].split(':')[0]

In [ ]:
thing.head()